In [1]:
from pathlib import Path
import polars as pl
from tqdm import tqdm
from rapidfuzz import process, fuzz

In [2]:
path = Path.home() / 'PycharmProjects/ML-learning-journey/nlp/text_retrieval/data/'
df = pl.read_csv(path / 'data.csv')
df

Line Number,Company Name,Company CIK Key,company_edit_name
i64,str,i64,str
1,"""!J INC""",1438823,"""!j inc"""
2,"""#1 A LIFESAFER…",1509607,"""#1 a lifesafer…"
3,"""#1 ARIZONA DIS…",1457512,"""#1 arizona dis…"
4,"""#1 PAINTBALL C…",1433777,"""#1 paintball c…"
5,"""$ LLC""",1427189,"""$ llc"""
6,"""& S MEDIA GROU…",1447162,"""& s media grou…"
7,"""&TV COMMUNICAT…",1479357,"""&tv communicat…"
8,"""'MKTG, INC.'""",886475,"""'mktg, inc.'"""
9,"""'OHANA LABS IN…",1703629,"""'ohana labs in…"


In [3]:
corpus = df['company_edit_name'].to_numpy().tolist()
result = process.extract('media group llc', corpus, scorer=fuzz.WRatio, limit=5)

In [4]:
# query = 'media group llc'
df_report = pl.DataFrame()
for query in tqdm(corpus[:100]):
    result = process.extract('media group llc', corpus, scorer=fuzz.WRatio, limit=5)
    df_result = (
        pl.DataFrame(result, schema=['result', 'score', 'index'])
        .with_columns(pl.lit(query).alias('query'))
    )
    df_report = df_report.vstack(df_result)
df_report

100%|██████████| 100/100 [00:07<00:00, 13.53it/s]


result,score,index,query
str,f64,i64,str
"""& s media grou…",95.0,5,"""!j inc"""
"""2am media grou…",95.0,2186,"""!j inc"""
"""amelia group l…",90.322581,26772,"""!j inc"""
"""4me group llc""",85.714286,3255,"""!j inc"""
"""#1 arizona dis…",85.5,2,"""!j inc"""
"""& s media grou…",95.0,5,"""#1 a lifesafer…"
"""2am media grou…",95.0,2186,"""#1 a lifesafer…"
"""amelia group l…",90.322581,26772,"""#1 a lifesafer…"
"""4me group llc""",85.714286,3255,"""#1 a lifesafer…"
